In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = pd.concat([train,test], axis = 0, ignore_index = True)
train =[]
test=[]

In [39]:
df.isnull().sum()

age                             0
antiguedad                      0
canal_entrada                   0
conyuemp                        0
fecha_alta                      0
fecha_dato                      0
ind_actividad_cliente           0
ind_ahor_fin_ult1          929615
ind_aval_fin_ult1          929615
ind_cco_fin_ult1           929615
ind_cder_fin_ult1          929615
ind_cno_fin_ult1           929615
ind_ctju_fin_ult1          929615
ind_ctma_fin_ult1          929615
ind_ctop_fin_ult1          929615
ind_ctpp_fin_ult1          929615
ind_deco_fin_ult1          929615
ind_dela_fin_ult1          929615
ind_deme_fin_ult1          929615
ind_ecue_fin_ult1          929615
ind_empleado                    0
ind_fond_fin_ult1          929615
ind_hip_fin_ult1           929615
ind_nom_pens_ult1          929832
ind_nomina_ult1                 0
ind_nuevo                       0
ind_plan_fin_ult1          929615
ind_pres_fin_ult1          929615
ind_reca_fin_ult1          929615
ind_recibo_ult

Impute Missing Values

In [6]:
dirtyrows = df[df.ind_empleado.isnull()].index

In [35]:
df = df.drop(dirtyrows, axis = 0)

In [7]:
#fill with most common registration method
df.canal_entrada = df.canal_entrada.fillna("Unknown")

In [8]:
#get rid of redundant province code column
df = df.drop("cod_prov", 1)

In [9]:
#Is spouse
df.conyuemp = df.conyuemp.fillna("Unknown")

In [10]:
#Customer type at the beginning of the month
df.loc[df.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'

df.loc[df.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'

df.loc[df.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'

df.loc[df.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'

In [11]:
df.indrel_1mes  = df.indrel_1mes.fillna('Primary')

In [12]:
#cutomer province
df.nomprov = df.nomprov.fillna("MADRID")

In [14]:
#gross household income
df.renta = df.renta.fillna(0)

In [16]:
df.loc[df.renta == '         NA',"renta"] = 0

In [ ]:
df.renta = df.renta.astype(float)

In [18]:
#impute missing incomes with median for province
df.loc[df.renta == 0, 'renta'] = df[df.renta > 0].groupby('nomprov').renta.transform('median')
df.loc[df.renta.isnull(), "renta"] = df.groupby('nomprov').renta.transform('median')

In [21]:
#impute segmento base on income brackets of the segmentos (type of account)
df.segmento = df[df.renta <= 96000].segmento.fillna("03 - UNIVERSITARIO")
df.segmento = df[df.renta <= 119500].segmento.fillna("02 - PARTICULARES")
df.segmento = df.segmento.fillna("01 - TOP")

In [23]:
#set the missing values of sex to most common
df.sexo = df.sexo.fillna("V")

In [24]:
#set missing values of customer relation at end of month
df.tiprel_1mes = df.tiprel_1mes.fillna('I')

In [25]:
#set missing value to most common for payroll and pensions products
df.ind_nomina_ult1 = df.ind_nomina_ult1.fillna(0.0)
df.ind_nom_pens_ult1ind_nom_  = df.ind_nom_pens_ult1.fillna(0.0)

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
# impute nonsensical values of seniority 
df.loc[df.antiguedad == -999999, 'antiguedad'] = df.antiguedad.median()

In [38]:
#change 99 to zero this is a binary variable for -is primary customer
df.loc[df.indrel == 99.0, "indrel"] = 0.0